In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from scipy.spatial.distance import cosine, cdist
from tqdm import tqdm

import scanpy as sc

import anndata as ad
import logging
import scgen

import os

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
basedir = '../l1000_data/'
train_data = pd.read_csv(basedir+'p1shp2xpr/training_data_p1.csv', index_col=None)
geneid = np.array(train_data.columns[2:]).astype('int')
train_data_sh = pd.read_csv(basedir+'p1shp2xpr/training_data_sh_p1.csv', index_col=None)
train_data2 = pd.read_csv(basedir+'p1shp2xpr/training_data_p2.csv', index_col=None)
train_data_ve = pd.read_csv(basedir+'p1shp2xpr/training_data_vehicle.csv', index_col=None)
train_data_ve = train_data_ve.loc[train_data_ve['pert_iname']=='DMSO']
train_data_ve.loc[:,'pert_iname'] = 'control'

basalccl = train_data.iloc[np.where(train_data['pert_iname']=='control')[0]]

train_data = pd.concat((train_data, train_data_sh, train_data2, train_data_ve))

val_data = pd.read_csv(basedir+'p1shp2xpr/validation_data_p1.csv', index_col=None)
val_data2 = pd.read_csv(basedir+'p1shp2xpr/validation_data_p2.csv', index_col=None)
val_data = pd.concat((val_data, val_data2))
infoidx = 2

# =============================================================================

train_data_info = train_data.iloc[:,:infoidx].values
train_data = train_data.iloc[:,infoidx:].values
val_data_info = val_data.iloc[:,:infoidx].values
val_data = val_data.iloc[:,infoidx:].values

geneinfo = pd.read_csv(basedir+'GSE92742_Broad_LINCS_gene_info.txt', sep='\t')
genemapper = pd.Series(data=geneinfo['pr_gene_symbol'].values, index=geneinfo['pr_gene_id'].values)
genesym = pd.Series(geneid).map(genemapper).values

mol_meta = pd.read_csv(basedir+'LINCS_small_molecules.tsv', sep='\t')
mol_meta.index = mol_meta['pert_name']
_, uid = np.unique(mol_meta.index, return_index=True)
mol_meta = mol_meta.iloc[uid]
mol_meta_tar = mol_meta.loc[mol_meta['target']!='-',:]

In [4]:
landmark_genes = pd.read_csv(basedir+'genelist.csv')
adata_train = ad.AnnData(pd.DataFrame(train_data, index=np.arange(train_data.shape[0]).astype('str'), columns=landmark_genes['pr_gene_symbol']))
adata_train.obs = pd.DataFrame(train_data_info, index=np.arange(train_data.shape[0]).astype('str'), columns=['sample', 'perturbation'])

In [5]:
totalpert = np.unique(train_data_info[:,1])
totalpert = totalpert[totalpert!='control']

In [6]:
scgen.SCGEN.setup_anndata(adata_train, batch_key="perturbation", labels_key="sample")

/data/yhhan/PGAN/scgen-env/lib/python3.9/site-packages/scvi/data/fields/_dataframe_field.py:183: UserWarning: Category 1321 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(


# Training

In [15]:
savedir = 'scgen_saved_models/model_perturbation_prediction.pt'
if os.path.exists(savedir+"/model.pt"):
    _model = torch.load(savedir+"/model.pt")
    model = scgen.SCGEN(adata_train)
    model.module.load_state_dict(_model['model_state_dict'])
else:
    model = scgen.SCGEN(adata_train)
    model.save(savedir, overwrite=True)
    model.train(
        max_epochs=100,
        batch_size=32,
        early_stopping=True,
        early_stopping_patience=25
    )
    model.save(savedir, overwrite=True)

# Chanye

In [17]:
adata_mc_landmark = sc.read_h5ad('ChangYe2021_SEACells.h5ad')
adata_mc_landmark

adata_mc_concat = adata_train.concatenate(adata_mc_landmark, batch_categories=['ref', 'new'])
adata_mc_concat

adata_mc_concat_combat = sc.pp.combat(adata_mc_concat, key='batch', inplace=False)

import qnorm
target_dist = np.median(adata_train.X, axis=0)
corrected_mc = qnorm.quantile_normalize(adata_mc_concat_combat[-adata_mc_landmark.shape[0]:,:].T, target=target_dist).T

Aalpha_ = corrected_mc[adata_mc_landmark.obs['perturbation']=='control']
Abeta_ = corrected_mc[adata_mc_landmark.obs['perturbation']=='erlotinib']

/tmp/ipykernel_15877/3999599176.py:4: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_mc_concat = adata_train.concatenate(adata_mc_landmark, batch_categories=['ref', 'new'])
/data/yhhan/PGAN/scgen-env/lib/python3.9/site-packages/scanpy/preprocessing/_combat.py:352: RuntimeWarning: divide by zero encountered in divide
  (abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max()


In [18]:
onlycont = corrected_mc[adata_mc_landmark.obs['perturbation'] == 'control']
adata_total_ = ad.AnnData(pd.DataFrame(onlycont, index=np.arange(onlycont.shape[0]).astype('str'), columns=landmark_genes['pr_gene_symbol']))
adata_total_.obs = adata_mc_landmark.obs[['sample', 'perturbation']][adata_mc_landmark.obs['perturbation'] == 'control']
adata_total_

AnnData object with n_obs × n_vars = 204 × 976
    obs: 'sample', 'perturbation'

In [19]:
adata_total = adata_train

In [20]:
# Include basal state to the adata_total
adata_total = adata_total.concatenate(adata_total_, batch_categories=['ref', 'new'])

/tmp/ipykernel_15877/1080020098.py:2: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_total = adata_total.concatenate(adata_total_, batch_categories=['ref', 'new'])


In [21]:
z,x,c = np.unique(adata_total.obs['sample'] + ' ' + adata_total.obs['perturbation'], return_counts=True, return_index=True)
adata_total = ad.concat((adata_total, adata_total[x[c==1]].copy()))

/data/yhhan/PGAN/scgen-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [22]:
scgen.SCGEN.setup_anndata(adata_total, batch_key="perturbation", labels_key="sample")

/data/yhhan/PGAN/scgen-env/lib/python3.9/site-packages/scvi/data/fields/_dataframe_field.py:183: UserWarning: Category 630 in adata.obs['_scvi_batch'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(


In [23]:
_model = torch.load(savedir+"/model.pt")
pred_model = scgen.SCGEN(adata_total)
pred_model.module.load_state_dict(_model['model_state_dict'])

<All keys matched successfully>

In [24]:
pred_model.is_trained=True

In [ ]:
pred_house = []
for p in totalpert:
    pred, _ = pred_model.predict(
        ctrl_key='control',
        stim_key=p,
        celltype_to_predict='SAMb6ba60b525'
    )
    pred.obs['condition'] = 'pred'

    pred_house.append(pred)

In [28]:
rankperf_house = []

mc_info = adata_mc_landmark.obs[['sample', 'perturbation']].values

cors_a_ = np.zeros(len(pred_house))
for p in range(len(pred_house)):
    cors_a_[p] = -cdist(corrected_mc[(mc_info[:,1]=='erlotinib'),:], pred_house[p].X, 'correlation').ravel().mean()
rankperf_house.append([cors_a_])

In [29]:
print(np.where(totalpert[np.argsort(rankperf_house[0][0])[::-1]]=='erlotinib')[0]+1, '/', totalpert.size,)

[11139] / 12794
